---
title: "Using the SPEX-Backend"
author: "Michael Lang"
date: "2025-01-18"
categories: [code]
format:
    html:
        code-fold: false
        eval: true
jupyter: blogqa
---

This tutorial will guide you through the usage of the **SPEX** backend for Tequila. 

We will cover:

- [What the SPEX backend is and why you might want to use it](#what-is-the-spex-backend)
- [Comparision with Qulacs](#comparision-with-qulacs)
- [Installation and basic setup](#installation-and-setup)
- [Computing expectation values using SPEX](#computing-expectation-values-using-spex)
- [How SPEX works](#how-spex-works)

If you're new to Tequila, we recommend starting with [Tequila Basics: Getting Started](../tq-get-started/index.qmd) to familiarize yourself with the core concepts.


## What is the SPEX backend?
**SPEX** is an expectation-value computation module for Tequila, implemented in C++ and specialized in sparse Pauli states. In typical quantum computations, a wavefunction can be written as

$$|\psi\rangle = \sum_{x=0}^{2^n - 1} c_x \, |x\rangle$$

When n is the number of qubits, and $c_x$ are complex amplitudes. Many simulation backends store all $2^n$ amplitudes, even if most of them are zero. In contrast, **SPEX** uses a **sparse representation**, keeping track only of non-zero amplitudes.

Therefore, **SPEX** can be advantageous for **sparsely occupied states**, e.g., states that are mostly zero except for a small number of basis states.

>Note: If your wavefunction is dense (i.e., almost all basis states have non-negligible amplitudes), then another simulator might be more straightforward. However SPEX will still function correctly.

At the end of the post, we will see a usecase from quantum chemistry where spex works quite nice.   

## Installation and Setup
First make sure the core Tequila package is installed:
```bash
pip install tequila-basic
```

Install the SPEX module:
```bash
pip install spex-tequila
```

Install on MacOS:  
The PyPi installation will work, but might struggle with using threads. A workaround is to install the gcc compiler suite (e.g. with homebrew) and then install spex directly from source
```bash
brew install gcc-14
CC=gcc-14 CXX=g++-14 pip install git+https://github.com/Mikel-Ma/spex
```

Make sure Tequila recognizes the SPEX backend:
```python
import tequila as tq
print(tq.available_simulators)
```

## Computing expectation values using SPEX
>Note: Because SPEX operates on exponential Pauli gates, it is most efficient if your circuit is already expressed using ExpPauli(...) gates. This way, you avoid extra compilation steps or transformations. If you use high-level gates like Rx, Ry, CNOT, and so on, Tequila will still convert them, but providing ExpPauli gates directly can eliminate unnecessary overhead.
```python
import tequila as tq
import numpy as np

U = tq.gates.ExpPauli(paulistring="X(0)Z(1)", angle=np.pi / 2)
H = tq.QubitHamiltonian("Z(0)")
E = tq.ExpectationValue(U=U, H=H)
```

Simulate with SPEX:
```python
result = tq.simulate(E, backend="spex")
print("Expectation value:", result)
```

### Using parallelization
You can control the number of threads by setting environment variables such as:

```bash
export SPEX_NUM_THREADS=4
```
or:
```bash
export OMP_NUM_THREADS=4
```

alternatively, you can use the argument *num_threads*:
```python
result = tq.simulate(E, backend="spex", num_threads=4)
```

If you do not set any of those variables, SPEX will use all available cores on your system.

### Setting Thresholds
The SPEX backend uses two important thresholds, an amplitude threshold and an angle threshold, to keep the simulation efficient by pruning negligible contributions. Both thresholds default to *1e-14*.

- Amplitude Threshold:
Any amplitude with an absolute value below this threshold is removed from the simulation. This ensures that only significant contributions are kept in the sparse state representation, reducing numerical noise and memory overhead.

- Angle Threshold:
If a gate’s rotation angle is below this threshold, the gate is ignored.

You can adjust these thresholds via the arguments *amplitude_threshold* and *angle_threshold*.

```python
result = tq.simulate(E, backend="spex", amplitude_threshold=1e-12, angle_threshold=1e-10)
```

>Note: If you want to disable pruning and use all elements, you can set the thresholds to *None*. Be aware setting the amplitude_threshold to *None* may significantly impact performance.

### Compressing Qubit Indicies
SPEX automatically compresses qubit indices to minimize state size. If you prefer to disable this behavior, use:
```python
result = tq.simulate(E, backend="spex", compress_qubits=False)
```

## How SPEX works
### State representation
SPEX represents quantum states as **sparse dictionaries** of non-zero amplitudes:
```python
|ψ⟩ = {
    0b00: (1.0 +0j), # |00⟩
    0b11: (0.0 -1j) # |11⟩
}
```

### Circuit implementation
In SPEX, a quantum circuit is a sequence of exponential Pauli gates. Each such gate can be thought of as an operator of the form:
$$e^{-i\theta P},$$

where $P$ is a product of Pauli matrices ($X$, $Y$, or $Z$).

When an exponential Pauli gate is applied, the simulator processes each non-zero amplitude in the dictionary. It first determines how $P$ flips or phases the corresponding basis state. Then, it computes updated amplitudes using:

$$\lvert\psi_{new}\rangle = \cos(\theta/2)\lvert\psi\rangle -i\sin(\theta/2)P\lvert\psi\rangle$$

>Note: By using fewer, larger Pauli exponentials, SPEX can process each gate in one pass over the sparse dictionary, minimizing overhead.

### Hamiltonian structure
Every Hamiltonian is expressed as:
$$H = \sum_{k} c_kP_k$$

Where each $P_k$ is a tensor product of Pauli matrices (e.g., $X \otimes Z$), and $c_k$ are (complex) coefficients.

### Term-wise expectation value calculation
To compute the expectation value $\langle \phi\lvert H\rvert \psi\rangle$, SPEX operates term by term:

1. **SPEX applies $P_k$ to $\lvert\psi\rangle$ to obtain $\lvert\psi_k\rangle$:**
    
    SPEX iterates over each non-zero basis state in $\lvert\psi\rangle$, depending on the Pauli operator:
    - **X**: Flips the qubits $\lvert0\rangle→\lvert1\rangle$
    - **Y**: Flips the qubit and multiplies by a phase $\pm i$
    - **Z**: Multiplies by $-1$ if the qubit is $\lvert1\rangle$

2. **SPEX computes the overlap $\langle\phi\lvert\psi_k\rangle$:**

    Using the same dictionary structure, SPEX finds matching basis states in $\lvert\phi\rangle$ and $\lvert\psi_k\rangle$ and sums up the product of their amplitudes.

3. **SPEX accumulates the result:**

    $$ \langle\phi\lvert P_k\rvert\psi\rangle\cdot c_k $$  
    This process is repeated for each term $c_kP_k$ and the results are summed, resulting in $\langle \phi\lvert H\rvert \psi\rangle$.


## Example

As an example, we show a full optimization of an orbital-optimized calculation of a VQE energy.  
The approach is described in more detail [here](https://arxiv.org/abs/2207.12421) (see code in the appendix)
- Ansatz: Separable Pairs, this is in principle classically simulable (and spex works ideal on this type of ansatz)
- HCB approximation: Makes things faster and doesn't loose accuracy for this ansatz
- Orbital-Optimization: Many VQEs are performed and the orbitals of the molecule are rotated until they give the lowest VQE energy.

Data below is on an apple M2 processor, with single-threaded spex

In [3]:
import tequila as tq
import time
import numpy

def run(n):
    
    geom = ""
    for k in range(2*n):
        geom += f"h 0.0 0.0 {1.5*k}\n"

    mol = tq.Molecule(geometry=geom, basis_set="sto-3g").use_native_orbitals()
    edges = [(2*i, 2*i+1) for i in range(n)]
    U = mol.make_ansatz(name="HCB-SPA", edges=edges)

    ig = numpy.eye(2*n)
    for i in range(n):
        ig[2*i][2*i+1] = 1.0
        ig[2*i+1][2*i] = -1.0

    start = time.time()
    opt = tq.chemistry.optimize_orbitals(molecule=mol, circuit=U, silent=True, initial_guess=ig.T, use_hcb=True, vqe_solver_arguments={"optimizer_arguments":{"backend":"spex"}})
    end = time.time()
    print(opt.iterations)
    print(mol.compute_energy("fci"))
    print(opt.energy)
    return end - start

for n in range(1,3):
    t = run(n)
    print("OO-SPA full optimization with {:3d} atoms took: {:2.4f}s".format(2*n,t))

2
-0.9981493534714088
-0.9108735545943865
OO-SPA full optimization with   2 atoms took: 8.4977s


/opt/anaconda3/envs/BlogQA/lib/python3.9/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute energy_nuc because it is not JSON-serializable
  warnings.warn(msg)


20
-1.996150325518811
-1.8291374124430246
OO-SPA full optimization with   4 atoms took: 38.1665s
